In [ ]:
! pip install transformers

In [ ]:
! pip install umap-learn

In [ ]:
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import os
from umap import UMAP

In [ ]:
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('AIRI-Institute/gena-lm-bert-large-t2t')
model = AutoModel.from_pretrained('AIRI-Institute/gena-lm-bert-large-t2t', output_hidden_states=True, trust_remote_code=True).to('cuda')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/46.0 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

modeling_bert.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/AIRI-Institute/gena-lm-bert-large-t2t:
- modeling_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=3)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (pre_attention_ln): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (post_attention_ln): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (s

In [ ]:
# define constants
num_chunks = 500
chunk_len = 5000
threshold = 1_000_000 # otherwise you may run out of RAM in Google Colab (adjustable)

In [ ]:
dir_content = os.listdir('/content/source_file')
needed_folders_names= list(filter(lambda f: f.endswith('.fasta'), dir_content))
needed_folders_names

['NC_022165.1.fasta',
 'NC_027458.1.fasta',
 'NC_031514.1.fasta',
 'NC_012145.1.fasta',
 'NC_022435.1.fasta',
 'NW_023500906.1.fasta',
 'NC_022161.1.fasta',
 'NC_027457.1.fasta',
 'NC_036379.1.fasta',
 'NC_036380.1.fasta',
 'NC_036376.1.fasta',
 'NC_013830.1.fasta',
 'NC_025331.1.fasta',
 'NC_006626.1.fasta',
 'NC_001224.1.fasta',
 'NC_018056.1.fasta',
 'NC_022170.1.fasta',
 'NC_022160.1.fasta',
 'NC_031515.1.fasta',
 'NC_022158.1.fasta',
 'NC_036378.1.fasta',
 'NC_013147.1.fasta',
 'NC_031511.1.fasta',
 'NC_003920.1.fasta',
 'NC_004918.1.fasta',
 'NC_012620.1.fasta',
 'NC_022173.1.fasta',
 'NC_006972.1.fasta',
 'NC_022172.1.fasta',
 'NC_022152.1.fasta',
 'NC_031184.1.fasta',
 'NC_006971.1.fasta',
 'NC_022174.1.fasta',
 'NC_016756.1.fasta',
 'NC_031185.1.fasta',
 'NC_022151.1.fasta',
 'NC_009638.1.fasta',
 'NC_014693.1.fasta',
 'NC_022162.1.fasta',
 'NC_031512.1.fasta',
 'NC_022153.1.fasta',
 'NC_011133.1.fasta',
 'NW_024066106.1.fasta',
 'NC_006077.1.fasta',
 'NC_022434.1.fasta',
 'NW

In [ ]:
splabels_1 = []
embeddings_1 = []
# Process each .fasta file
for sl, fn in enumerate(needed_folders_names):
    full_path = os.path.join('/content/source_file', fn)
    print(f'Processing ({sl}): {full_path}')
     # assemble all chromosomes into one sequence (very long)
    with open(full_path, 'r') as f:
        all_lines = f.readlines()[:threshold]
        all_lines_cleared = []
        for l in all_lines:
            if not l.startswith('>'):
                all_lines_cleared.append(l.strip().upper())

    final_one_line = ''.join(all_lines_cleared)

    array_of_chunks = []  # You can proceed with your chunk logic here
     # get random subsequences from that one very long sequence
    for i in range(num_chunks):
        n = np.random.randint(0, len(final_one_line) - chunk_len)
        array_of_chunks.append(final_one_line[n:n + chunk_len])

    flag = True
    sequence_embeddings = []

    # tokenize obtained subsequences and construct appropriate dict() for GENA model
    for i in range(num_chunks):
        tokenized_seq = tokenizer(array_of_chunks[i], max_length=512, padding="max_length", truncation="longest_first")
        tokenized_seq['input_ids'] = torch.tensor([tokenized_seq['input_ids']]).to('cuda')
        tokenized_seq['token_type_ids'] = torch.tensor([tokenized_seq['token_type_ids']]).to('cuda')
        tokenized_seq['attention_mask'] = torch.tensor([tokenized_seq['attention_mask']]).to('cuda')

        # get embeddings
        output_hs = model(**tokenized_seq).hidden_states

        if flag:
            print(f'Length of the full output: {len(output_hs)}')
            print(f'Shape of the output of the last layer: {output_hs[-1].shape}')
            flag = False
         # Append embeddings from the chunk to the list for averaging
        sequence_embeddings.append(output_hs[-1].squeeze().detach().cpu().numpy())


    # Average the embeddings of all chunks to get a single embedding per sequence
    final_embedding = np.mean(sequence_embeddings, axis=0)

    # Store the final embedding and the corresponding label
    embeddings_1.append(final_embedding)
    splabels_1.append(sl)

Processing (0): /content/source_file/NC_022165.1.fasta


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1735: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Length of the full output: 25
Shape of the output of the last layer: torch.Size([1, 512, 1024])
Processing (1): /content/source_file/NC_027458.1.fasta
Length of the full output: 25
Shape of the output of the last layer: torch.Size([1, 512, 1024])
Processing (2): /content/source_file/NC_031514.1.fasta
Length of the full output: 25
Shape of the output of the last layer: torch.Size([1, 512, 1024])
Processing (3): /content/source_file/NC_012145.1.fasta
Length of the full output: 25
Shape of the output of the last layer: torch.Size([1, 512, 1024])
Processing (4): /content/source_file/NC_022435.1.fasta
Length of the full output: 25
Shape of the output of the last layer: torch.Size([1, 512, 1024])
Processing (5): /content/source_file/NW_023500906.1.fasta
Length of the full output: 25
Shape of the output of the last layer: torch.Size([1, 512, 1024])
Processing (6): /content/source_file/NC_022161.1.fasta
Length of the full output: 25
Shape of the output of the last layer: torch.Size([1, 512, 10

In [ ]:
import pandas as pd
result=np.stack(embeddings_1)
result.shape
np.save('result.npy', result)

In [ ]:
load=np.load('result.npy')

In [ ]:
load.shape

(64, 512, 1024)

In [ ]:
final_embedding = np.mean(sequence_embeddings, axis=1)

    # Store the final embedding and the corresponding label
embeddings_1.append(final_embedding)
splabels_1.append(sl)

In [ ]:
final_embedding.shape

(500, 1024)